# Explore SMPL and SMPL-X

references
- [smpl-x github](https://github.com/vchoutas/smplx)

In [4]:
import os

os.chdir("/root/dev/human/")
os.getcwd()

'/root/dev/human'

In [5]:
import torch

torch.cuda.set_device(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
!gpustat

dc603d09d8db                Mon Sep  2 23:42:30 2024  550.100
[0] NVIDIA GeForce RTX 4090 | 39°C,   0 % |   740 / 24564 MB |
[1] NVIDIA GeForce RTX 4090 | 36°C,   0 % |    21 / 24564 MB |


## Exploration of SMPL

In [57]:
!ls /root/dev/playground/dataset/smpl/models

basicmodel_f_lbs_10_207_0_v1.1.0.pkl
basicmodel_m_lbs_10_207_0_v1.1.0.pkl
basicmodel_neutral_lbs_10_207_0_v1.1.0.pkl


In [76]:
# pickle file
import pickle
from typing import List

# with open(input_models[0], "rb") as body_file:
#     unpickled = pickle._Unpickler(body_file)
#     unpickled.encoding = "latin1"
#     body_data = unpickled.load()
with open("/root/dev/playground/dataset/smpl/models/basicmodel_f_lbs_10_207_0_v1.1.0.pkl", "rb") as body_file:
    unpickled = pickle._Unpickler(body_file)
    unpickled.encoding = "latin1"
    body_data = unpickled.load()

for key in sorted(body_data.keys()):
    value = body_data[key]
    if isinstance(value, List):
        print(f"{key}: List")
        print(f"len: {len(value)}")
        print(f"[0]: {value[0].shape}")
    elif isinstance(value, str):
        print(f"{key}: str")
        print(value)
    else:
        print(f"{key}: {value.shape}")

J: (24, 3)
J_regressor: (24, 6890)
J_regressor_prior: (24, 6890)
bs_style: str
lbs
bs_type: str
lrotmin
f: (13776, 3)
kintree_table: (2, 24)
posedirs: (6890, 3, 207)
shapedirs: (6890, 3, 300)
v_template: (6890, 3)
vert_sym_idxs: (6890,)
weights: (6890, 24)
weights_prior: (6890, 24)


## Exploration of SMPL-X

In [71]:
!ls /root/dev/playground/dataset/smplx

SMPLX_FEMALE.npz  SMPLX_MALE.npz  SMPLX_NEUTRAL.npz  smplx_npz.zip
SMPLX_FEMALE.pkl  SMPLX_MALE.pkl  SMPLX_NEUTRAL.pkl  version.txt


In [90]:
from typing import List

import numpy as np

body_data = np.load("/root/dev/playground/dataset/smplx/SMPLX_MALE.npz", allow_pickle=True)

# print(body_data.keys())

for key in sorted(body_data.keys()):
    value = body_data[key]
    if isinstance(value, np.ndarray):
        print(f"{key}: {value.shape}")
    else:
        print(f"{key}: {value.shape}")

J_regressor: (55, 10475)
allow_pickle: ()
dynamic_lmk_bary_coords: (79, 17, 3)
dynamic_lmk_faces_idx: (79, 17)
f: (20908, 3)
ft: (20908, 3)
hands_coeffsl: (1554, 45)
hands_coeffsr: (1554, 45)
hands_componentsl: (45, 45)
hands_componentsr: (45, 45)
hands_meanl: (45,)
hands_meanr: (45,)
joint2num: ()
kintree_table: (2, 55)
lmk_bary_coords: (51, 3)
lmk_faces_idx: (51,)
part2num: ()
posedirs: (10475, 3, 486)
shapedirs: (10475, 3, 400)
v_template: (10475, 3)
vt: (11313, 2)
weights: (10475, 55)


In [89]:
from typing import List

import pickle

with open("/root/dev/playground/dataset/smplx/SMPLX_MALE.pkl", "rb") as body_file:
    unpickled = pickle._Unpickler(body_file)
    unpickled.encoding = "latin1"
    body_data = unpickled.load()

# print(body_data.keys())

for key in sorted(body_data.keys()):
    value = body_data[key]
    if isinstance(value, List):
        print(f"{key}: List")
        print(f"len: {len(value)}")
        print(f"[0]: {value[0].shape}")
    else:
        print(f"{key}: {value.shape}")

J_regressor: (55, 10475)
dynamic_lmk_bary_coords: List
len: 79
[0]: (17, 3)
dynamic_lmk_faces_idx: (79, 17)
f: (20908, 3)
ft: (20908, 3)
hands_coeffsl: (1554, 45)
hands_coeffsr: (1554, 45)
hands_componentsl: (45, 45)
hands_componentsr: (45, 45)
hands_meanl: (45,)
hands_meanr: (45,)
joint2num: ()
kintree_table: (2, 55)
lmk_bary_coords: (51, 3)
lmk_faces_idx: (51,)
part2num: ()
posedirs: (10475, 3, 486)
shapedirs: (10475, 3, 400)
v_template: (10475, 3)
vt: (11313, 2)
weights: (10475, 55)


## Preprocessing
- Before using SMPL and SMPL+H you should follow the instructions below to remove the Chumpy objects from both model pkls, as well as merge the MANO parameters with SMPL+H.

In [13]:
""" tools/clean_ch.py """

from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

import os
import argparse

import numpy as np
import pickle
from tqdm import tqdm


def clean_fn(fn, output_folder='output'):
    with open(fn, 'rb') as body_file:
        unpickled = pickle._Unpickler(body_file)
        unpickled.encoding = "latin1"
        body_data = unpickled.load()

    output_dict = {}
    for key, data in body_data.iteritems():
        if 'chumpy' in str(type(data)):
            output_dict[key] = np.array(data)
        else:
            output_dict[key] = data

    out_fn = os.path.split(fn)[1]

    out_path = os.path.join(output_folder, out_fn)
    with open(out_path, 'wb') as out_file:
        pickle.dump(output_dict, out_file)

In [14]:
!ls /root/dev/playground/dataset/smplx

SMPLX_FEMALE.npz  SMPLX_MALE.npz  SMPLX_NEUTRAL.npz  smplx_npz.zip
SMPLX_FEMALE.pkl  SMPLX_MALE.pkl  SMPLX_NEUTRAL.pkl  version.txt


In [15]:
input_models = ["/root/dev/playground/dataset/smplx/SMPLX_MALE.pkl",
                "/root/dev/playground/dataset/smplx/SMPLX_FEMALE.pkl",
                "/root/dev/playground/dataset/smplx/SMPLX_NEUTRAL.pkl"] # SMPLX_MALE, SMPLX_FEMALE, SMPLX_NEUTRAL
output_folder = "/root/dev/playground/dataset/smplx_processed"
if not os.path.exists(output_folder):
    print(f"Creating directory: {output_folder}")
    os.makedirs(output_folder)
for input_model in input_models:
    clean_fn(input_model, output_folder=output_folder)

AttributeError: 'dict' object has no attribute 'iteritems'

- body data in SMPL-X

In [17]:
with open(input_models[0], "rb") as body_file:
    unpickled = pickle._Unpickler(body_file)
    unpickled.encoding = "latin1"
    body_data = unpickled.load()

print(body_data.keys())

dict_keys(['dynamic_lmk_bary_coords', 'hands_componentsl', 'ft', 'lmk_faces_idx', 'f', 'J_regressor', 'hands_componentsr', 'kintree_table', 'hands_coeffsr', 'joint2num', 'hands_meanl', 'lmk_bary_coords', 'weights', 'posedirs', 'dynamic_lmk_faces_idx', 'part2num', 'vt', 'hands_meanr', 'hands_coeffsl', 'v_template', 'shapedirs'])


In [51]:
from typing import List

for key in sorted(body_data.keys()):
    value = body_data[key]
    if isinstance(value, List):
        print(f"{key}: List")
        print(f"len: {len(value)}")
        print(f"[0]: {value[0].shape}")
        print()
    else:
        print(f"{key}: {value.shape}")
        print()

J_regressor: (55, 10475)

dynamic_lmk_bary_coords: List
len: 79
[0]: (17, 3)

dynamic_lmk_faces_idx: (79, 17)

f: (20908, 3)

ft: (20908, 3)

hands_coeffsl: (1554, 45)

hands_coeffsr: (1554, 45)

hands_componentsl: (45, 45)

hands_componentsr: (45, 45)

hands_meanl: (45,)

hands_meanr: (45,)

joint2num: ()

kintree_table: (2, 55)

lmk_bary_coords: (51, 3)

lmk_faces_idx: (51,)

part2num: ()

posedirs: (10475, 3, 486)

shapedirs: (10475, 3, 400)

v_template: (10475, 3)

vt: (11313, 2)

weights: (10475, 55)

